# Tutorial 2

In [1]:
pip install quimb

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install numba

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install scipy

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install numpy

Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install cytoolz

Note: you may need to restart the kernel to use updated packages.


In [6]:
pip install tqdm

Note: you may need to restart the kernel to use updated packages.


In [7]:
pip install psutil

Note: you may need to restart the kernel to use updated packages.


In [8]:
pip install opt_einsum

Note: you may need to restart the kernel to use updated packages.


In [9]:
pip install autoray

Note: you may need to restart the kernel to use updated packages.


In [10]:
pip install matplotlib

Note: you may need to restart the kernel to use updated packages.


In [11]:
pip install networkx

Note: you may need to restart the kernel to use updated packages.


In [12]:
import quimb as qu
from quimb import *
import quimb.tensor as qtn
import numpy as np

## Basic Operations
The $\dagger$ (‘dagger’), or hermitian conjugate, operation is performed with the .H attribute:

In [13]:
psi = 1.0j * bell_state('psi-')
print(psi)

[[ 0.+0.j      ]
 [ 0.+0.707107j]
 [-0.-0.707107j]
 [ 0.+0.j      ]]


In [14]:
psi.H

[[ 0.-0.j        0.-0.707107j -0.+0.707107j  0.-0.j      ]]

This takes the ket-vector to a bra-vector:

$ | \psi \rangle \mapsto \langle \psi | $

Numerically, this is just the complex conjugate of the column vector $| \psi \rangle$:

$\begin{pmatrix}0\\ 0.707107j\\ -0.707107j\\  0 \end{pmatrix} \mapsto \begin{pmatrix}0,\ -0.707107j,\ 0.707107j,\  0 \end{pmatrix}$

This can also be performed on operators of course, where it is typically referred to as the Hermitian conjugate (denoted by $\dagger$), and is given by the complex conjugate of the transpose of the matrix representing the operator:

In [15]:
psi_op = qu(psi, qtype='dop')
print(psi_op)

[[ 0. +0.j  0. +0.j -0. +0.j  0. +0.j]
 [ 0. +0.j  0.5+0.j -0.5+0.j  0. +0.j]
 [ 0. -0.j -0.5-0.j  0.5+0.j  0. -0.j]
 [ 0. +0.j  0. +0.j -0. +0.j  0. +0.j]]


In [16]:
psi_op.H

[[ 0. -0.j  0. -0.j  0. +0.j  0. -0.j]
 [ 0. -0.j  0.5-0.j -0.5+0.j  0. -0.j]
 [-0. -0.j -0.5-0.j  0.5-0.j -0. -0.j]
 [ 0. -0.j  0. -0.j  0. +0.j  0. -0.j]]

This takes the operator $ U = |\psi \rangle \langle \psi |$ to the operator $U^{\dagger}$

This is just the combination of ```.conj()``` and ```.T```, but only available for ```scipy.sparse``` matrices and ```qarray```s (not ```numpy.ndarray```s).

The product of two quantum objects is the dot or matrix product, which, since python 3.5, has been overloaded with the ```@``` symbol. Using it is recommended:

In [17]:
psi = up() #spin-up single qubit
psi

[[1.+0.j]
 [0.+0.j]]

In [18]:
psi.H @ psi  # inner product

[[1.+0.j]]

In [19]:
X = pauli('X')
X @ psi  # act as gate

[[0.+0.j]
 [1.+0.j]]

In [20]:
psi.H @ X @ psi  # operator expectation

[[0.+0.j]]

Scalar expectation values might best be computed using the ```expectation()``` function (aliased to ```expec()```) which dispatches to accelerated methods:

In [21]:
expec(psi, psi)

1.0

In [22]:
expec(psi, X)

0j

Here’s an example for a much larger (20 qubit), sparse operator expecation, which will be **automatically parallelized**:

In [23]:
psi = rand_ket(2**20)
A = rand_herm(2**20, sparse=True) + speye(2**20)
A

<1048576x1048576 sparse matrix of type '<class 'numpy.complex128'>'
	with 11534278 stored elements in Compressed Sparse Row format>

In [24]:
expec(A, psi)  # should be ~ 1

/Users/amelieschreiber/anaconda3/lib/python3.7/site-packages/quimb/core.py:508: NumbaWarning: Cannot cache compiled function "_dot_csr_matvec_prange" as it uses dynamic globals (such as ctypes pointers and large global arrays)
  @njit(parallel=True)  # pragma: no cover


0.9993532985313052

In [25]:
%%timeit
expec(A, psi)

165 ms ± 14.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## Combining Objects - Tensoring

There are a number of ways to combine states and operators, i.e. tensoring them together.

Functional form using kron():

```
kron(psi1, psi2, psi3, ...)
```

This can also be done using the & overload on qarray and scipy matrices:

```
psi1 & psi2 & psi3
```

### Warning

When quimb is imported, it monkey patches the otherwise unused method of ```&/__and__``` of scipy sparse matrices to ```kron()```.

Often one wants to sandwich an operator with many identities, ```ikron()``` can be used for this:

In [26]:
dims = [2] * 10  # overall space of 10 qubits
X = pauli('X')
IIIXXIIIII = ikron(X, dims, inds=[3, 4])  # act on 4th and 5th spin only
IIIXXIIIII.shape

/Users/amelieschreiber/anaconda3/lib/python3.7/site-packages/quimb/core.py:739: NumbaWarning: Cannot cache compiled function "_nb_kron_exp_par" as it uses dynamic globals (such as ctypes pointers and large global arrays)
  @njit(parallel=True)


(1024, 1024)

For more advanced tensor constructions, such as reversing and interleaving identities within operators ```pkron()``` can be used:

In [27]:
dims = [2] * 3
XZ = pauli('X') & pauli('Z')
ZIX = pkron(XZ, dims, inds=[2, 0])
ZIX.real.astype(int)

[[ 0  1  0  0  0  0  0  0]
 [ 1  0  0  0  0  0  0  0]
 [ 0  0  0  1  0  0  0  0]
 [ 0  0  1  0  0  0  0  0]
 [ 0  0  0  0  0 -1  0  0]
 [ 0  0  0  0 -1  0  0  0]
 [ 0  0  0  0  0  0  0 -1]
 [ 0  0  0  0  0  0 -1  0]]

$ZIX = Z \otimes I \otimes X$ would then act with Z on first spin, and X on 3rd.

## Removing Objects - Partial Trace
To remove, or ignore, certain parts of a quantum state the partial trace function ```partial_trace()``` (aliased to ```ptr()```) is used. Here, the internal dimensions of a state must be supplied as well as the indicies of which of these subsystems to keep.

For example, if we have a random system of 10 qubits (hilbert space of dimension $2^{10}$), and we want just the reduced density matrix describing the first and last spins:

In [28]:
dims = [2] * 10
D = prod(dims)
psi = rand_ket(D)
rho_ab = ptr(psi, dims, [0, 9])
rho_ab.round(3)  # probably pretty close to identity

[[ 0.241+0.j     0.002-0.009j -0.008-0.003j  0.002-0.01j ]
 [ 0.002+0.009j  0.234+0.j     0.01 +0.012j  0.003+0.001j]
 [-0.008+0.003j  0.01 -0.012j  0.265+0.j    -0.002+0.007j]
 [ 0.002+0.01j   0.003-0.001j -0.002-0.007j  0.26 +0.j   ]]

```partial_trace()``` accepts dense or sparse, operators or vectors.